In [3]:
# !pip install git+https://hf.co/ctheodoris/Geneformer.git
# !pip install anndata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 4.5 MB/s eta 0:00:00


In [5]:
from geneformer.pretrainer import token_dictionary

In [53]:
from geneformer.tokenizer import GENE_MEDIAN_FILE
import pickle


with open(GENE_MEDIAN_FILE, "rb") as f:
        gene_median_dict = pickle.load(f)

In [6]:
import pandas as pd

sequence_path = '/home/ec2-user/cytoself-data/sequences.csv'
sequences = pd.read_csv(sequence_path, index_col=0)

In [7]:
sequences

,Peptide,localization,loc
ensg,,,
ENSG00000075624,MDDDIAALVVDNGSGMCKAGFAGDDAPRAVFPSIVGRPRHQGVMVG...,other,8
ENSG00000100591,MAKWGEGDPRWIVEERADATNVNNWHWTERDASNWSTDKLKTLFLA...,other,8
ENSG00000156735,MSALRRSGYGPSDGPSYGRYYGPGGGDVPVHPPPPLYPLRPEPPQP...,other,8
ENSG00000229524,MEPNDSTSTAVEEPDSLEVLVKTLDSQTRTFIVGAQMNVKEFKEHI...,other,8
ENSG00000105401,MVDYSVWDHIEVSDDEDETHPNIDTASLFRWRHQARVERMEQFQKE...,other,8
...,...,...,...
ENSG00000174749,MCSAGELLRGGDGGERDEDGDALAEREAAGTGWDPGASPRRRGQRP...,ER,0
ENSG00000138439,MSQRVRRNGSPTPAGSLGGGAVATAGGPGSRLQPMRATVPFQLKQQ...,other,8
ENSG00000116001,MEDEMPKTLYVGNLSRDVTEALILQLFSQIGPCKNCKMIMDTAGND...,other,8


In [37]:
tokens = [token_dictionary[seq] if seq in token_dictionary else 1 for seq in sequences.index]
missing = [seq for seq in sequences.index if seq not in token_dictionary.keys()]

In [39]:
from transformers import AutoModelForMaskedLM

In [40]:
model = AutoModelForMaskedLM.from_pretrained("ctheodoris/Geneformer")
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(25426, 256, padding_idx=0)
      (position_embeddings): Embedding(2048, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.02, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.02, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine

In [24]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(25426, 256, padding_idx=0)
      (position_embeddings): Embedding(2048, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.02, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.02, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine

In [41]:
import torch

tokens = torch.asarray(tokens).unsqueeze(0)
embeddings = model.bert.embeddings(tokens)
embeddings = embeddings.squeeze(0).detach().numpy()

In [15]:
# import numpy as np

# geneformer_embeddings_path = '/home/ec2-user/cytoself-data/geneformer_embeds.npy'
# np.save(geneformer_embeddings_path, embeddings)

## Using abundance too

In [16]:
abundance_path = '/home/ec2-user/cytoself-data/opencell-protein-abundance.csv'

abundance = pd.read_csv(abundance_path)

In [34]:
model.bert.forward?

Signature:
model.bert.forward(
    input_ids: Optional[torch.Tensor] = None,
    attention_mask: Optional[torch.Tensor] = None,
    token_type_ids: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.Tensor] = None,
    head_mask: Optional[torch.Tensor] = None,
    inputs_embeds: Optional[torch.Tensor] = None,
    encoder_hidden_states: Optional[torch.Tensor] = None,
    encoder_attention_mask: Optional[torch.Tensor] = None,
    past_key_values: Optional[List[torch.FloatTensor]] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple[torch.Tensor], transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions]
Docstring:
The [`BertModel`] forward method, overrides the `__call__` special method.

<Tip>

Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
instance af

In [36]:
tokens.shape

torch.Size([1, 1, 1311])

In [51]:
output = model.bert(input_ids=tokens).last_hidden_state.squeeze(0)

In [52]:
output.shape

torch.Size([1311, 256])

In [18]:
model.forward?

Signature:
model.forward(
    input_ids: Optional[torch.Tensor] = None,
    attention_mask: Optional[torch.Tensor] = None,
    token_type_ids: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.Tensor] = None,
    head_mask: Optional[torch.Tensor] = None,
    inputs_embeds: Optional[torch.Tensor] = None,
    encoder_hidden_states: Optional[torch.Tensor] = None,
    encoder_attention_mask: Optional[torch.Tensor] = None,
    labels: Optional[torch.Tensor] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple[torch.Tensor], transformers.modeling_outputs.MaskedLMOutput]
Docstring:
The [`BertForMaskedLM`] forward method, overrides the `__call__` special method.

<Tip>

Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
instance afterwards instead of this since the former takes care of running the pre and post proces